In [ ]:
#import libraries for scraping
import pandas as pd
from bs4 import BeautifulSoup
import requests
import schedule
import time
import mysql.connector

In [11]:
#connecting to database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="science1",
  database="news",
  charset="utf8mb4"
)
mycursor = mydb.cursor()

In [12]:
# the main functions for scraping different several websites
data_all = []
def scrape_techcrunch(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    latest = bs.find('div', attrs={'class': 'wp-block-group is-layout-flow wp-block-group-is-layout-flow'})
    writer = latest.find_all('div', attrs={'class': 'wp-block-tc23-author-card-name has-xsmall-font-size wp-elements-d055c9e8dc404f1acd88f01867eaea50'})
    title = latest.find_all('div', attrs={'class': 'wp-block-tc23-post-picker'})
    link = latest.find_all('div', attrs={'class': 'wp-block-tc23-post-picker'})
    photo_link = latest.find_all('figure', attrs={'class': 'wp-block-post-featured-image'})
    category = latest.find_all('a', attrs={'class': 'is-taxonomy-category has-text-color has-green-900-color wp-block-tenup-post-primary-term has-small-font-size wp-elements-da2f4b8fa1d086a1fe3f2c2891f7e77e'})
    date = latest.find_all('time', attrs={'class': 'has-text-color has-grey-500-color wp-block-tc23-post-time-ago has-xsmall-font-size'})
    
    list_len = min(
        len(writer),
        len(title),
        len(link),
        len(photo_link),
        len(category),
        len(date)
    )
    for i in range(list_len):       
        data_all.append({
            'link': link[i].find('h2').find('a').get('href'),
            'writer': writer[i].find('a').get_text(),
            'title': title[i].find('h2').find('a').get_text(),
            'photo_link': photo_link[i].find('img').get('src'),
            'category': category[i].get_text().strip(),
            'date': date[i].get_text().strip()
        })
def scrape_techrepublic(url):
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    latest = bs.find('section',attrs = {'class':'list-featured list-external'})
    
    writer_republic =  latest.find_all('span',attrs = {'property':'name'})
    title_republic = latest.find_all('h3',attrs = {'class':'headline'})
    link_republic = latest.find_all('h3',attrs = {'class': 'headline'})
    photo_link_republic = latest.find_all('img',attrs = {'class':'attachment-thumbnail-md size-thumbnail-md'})
    catagory_republic = latest.find_all('h4',attrs = {'class':'category-title'})
    date_republic = latest.find_all('span',attrs = {'class':'date-published'})
    list_len = min(
        len(writer_republic),
        len(title_republic),
        len(link_republic),
        len(photo_link_republic),
        len(catagory_republic),
        len(date_republic)
    )

    for i in range(list_len):
       data_all.append({
            'link':link_republic[i].find('a').get('href'),
            'writer': writer_republic[i].get_text().strip().split('     ')[-1].strip(),
            'title':title_republic[i].find('a').get_text().strip(),
            'photo_link': photo_link_republic[i].get('src'),
            'category':catagory_republic[i].get_text().strip(),
            'date': date_republic[i].find('time').get_text().strip()
        })
    return data_all   

In [13]:
#calling main functions to scrape websites using there url's 
def url():
    global link_all
    paypoll = 'https://www.techrepublic.com/topic/payroll/'
    developer = 'https://www.techrepublic.com/topic/developer/'
    AI = 'https://www.techrepublic.com/topic/artificial-intelligence/'
    security = 'https://www.techrepublic.com/topic/security/'
    main_techcrunch = 'https://techcrunch.com/'
    
    paypoll = scrape_techrepublic(paypoll)
    developer = scrape_techrepublic(developer)
    AI = scrape_techrepublic(AI)
    security = scrape_techrepublic(security)
    current_url = scrape_techcrunch(main_techcrunch)
url()

In [14]:
content_all = {}

def content(url):
    # بررسی اینکه آیا URL قبلاً پردازش شده است یا خیر
    if url in content_all:
        print(f'URL {url} قبلاً پردازش شده است.')
        return

    # درخواست به URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # استخراج محتوای مورد نظر
    techcrunch_content = soup.find_all('div', attrs={'class': 'entry-content wp-block-post-content is-layout-flow wp-block-post-content-is-layout-flow'})
    techrepublic_content = soup.find_all('section',attrs = {'class':'article-body'})
    # اضافه کردن محتوا به دیکشنری با استفاده از URL به عنوان کلید
    content_all[url] = [techcrunch_content[i].get_text().strip() for i in range(len(techcrunch_content))]
    content_all[url] = [techrepublic_content[i].get_text() for i in range(len(techrepublic_content))]

    print('scrape url is finished')  

In [ ]:
# getting url of every article and get there content
def main_content():
    try:
        # Iterate over each dictionary in the data_all list
        for item in data_all:
            # Extract the 'link' from each dictionary
            url = item.get('link')
            # Pass the URL to the content function if it's not None
            if url:
                content(url)
            else:
                print("No link found in item.")
    except AttributeError:
        print('An attribute error occurred.')
    except Exception as e:
        print(f'An error occurred: {e}')

main_content()

In [ ]:
# save obtained data in database
def save_to_db():
    for entry in data_all:
        # کوئری درج داده‌ها
        sql = '''INSERT IGNORE INTO scraper (title, link, writer, date, photo_link, category)
                 VALUES (%s, %s, %s, %s, %s, %s)'''
        val = (entry['title'], entry['link'], entry['writer'], entry['date'], entry['photo_link'], entry['category'])
        mycursor.execute(sql, val)
    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

save_to_db()

In [21]:
schedule.every(1).minutes.do(url)
schedule.every(1).minutes.do(main_content)
schedule.every(1).minutes.do(save_to_db)

# Continuously run the scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)